In [ ]:
#Citation for the below code
'''
Title: UAV LiCNN Tile Creation
Authors: Galen Richardson, Krishan Rajaratnam, Julie Lovitt
Date: 07-14-2021
Version: 1
https://www.nrcan.gc.ca/science-and-data/research-centres-and-labs/canada-centre-remote-sensing/21749
'''

In [1]:
from pathlib import Path
import cv2
import math
import numpy as np
import PIL,sys,shutil,os
from PIL import Image
import tensorflow as tf
#for dislaying results
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
plt.rcParams['figure.dpi'] = 150 
%matplotlib inline
import time

In [14]:
project_dir = Path("").resolve()
Photo_dir = str(project_dir / 'data/raw/test')
Mask_dir = str(project_dir / 'data/raw/test_mask')
out_photo_dir = str(project_dir / 'data/raw/test_3x3')
out_mask_dir = str(project_dir / 'data/raw/test_mask3x3')
augments=True

In [15]:
def dirs_setup(Photo_dir,Mask_dir):
    global photo_list,mask_list,file_list
    file_list= []
    photo_list=[]
    mask_list=[]
    for path,subdir,files in os.walk(Photo_dir):
        for file in files:
            file_list.append(file)
            photo_list.append(Photo_dir+'\\'+file)
            mask_list.append(Mask_dir+'\\'+file)
    if len(photo_list)!=len(mask_list):
        print('input lists are different lenghts!!!')


In [16]:
dirs_setup(Photo_dir,Mask_dir)

In [17]:
import random
rand_list=[]
for i in range(200):
    x=random.sample(range(len(mask_list)), 9)
    if x in rand_list:
        print('dup please redo')
        
        continue
    else:
        rand_list.append(x)
        
    

In [18]:
iter_num=0
for rand in rand_list:
    top_images = [Image.open(x) for x in [photo_list[rand[0]], photo_list[rand[1]],photo_list[rand[2]]]]
    mid_images = [Image.open(x) for x in [photo_list[rand[3]], photo_list[rand[4]],photo_list[rand[5]]]]
    bot_images = [Image.open(x) for x in [photo_list[rand[6]], photo_list[rand[7]],photo_list[rand[8]]]]
    photo_im = Image.new('RGB', (240, 240))
    t_offset = 0
    m_offset = 0
    b_offset = 0
    for im in top_images:
        photo_im.paste(im, (t_offset,0))
        t_offset += im.size[0]
    for im in mid_images:
        photo_im.paste(im, (m_offset,80))
        m_offset += im.size[0]
    for im in bot_images:
        photo_im.paste(im, (b_offset,160))
        b_offset += im.size[0]
    if augments==True:
        num2 = tf.random.uniform(())
        num3 = tf.random.uniform(())
        num4 = tf.random.uniform(())
        num5 = tf.random.uniform(()) 
        tf_img=tf.keras.preprocessing.image.img_to_array(photo_im)
        tf_img = tf.cast(tf_img, tf.float32) / 255.0
        if num4>.75:
            s_img= tf.image.random_saturation(tf_img, 0.5,1)
            c_img= tf.image.random_contrast(s_img, 0.5,.9)
            tf_img = tf.clip_by_value(c_img, clip_value_min=0., clip_value_max=1.)
        if num4<.25:
            c_img= tf.image.random_contrast(tf_img, 0.5,1)
            tf_img = tf.clip_by_value(c_img, clip_value_min=0., clip_value_max=1.)
        if num3<.25:
            b_img=tf.image.adjust_brightness(tf_img, -.1)
            tf_img = tf.clip_by_value(b_img, clip_value_min=0., clip_value_max=1.)
        tf.keras.preprocessing.image.save_img(out_photo_dir+'\\'+str(iter_num)+'.png', tf_img, data_format=None, file_format=None, scale=True,)
    else:
        photo_im.save(out_photo_dir+'\\'+str(iter_num)+'.png')
        
    top_images = [Image.open(x) for x in [mask_list[rand[0]], mask_list[rand[1]],mask_list[rand[2]]]]
    mid_images = [Image.open(x) for x in [mask_list[rand[3]], mask_list[rand[4]],mask_list[rand[5]]]]
    bot_images = [Image.open(x) for x in [mask_list[rand[6]], mask_list[rand[7]],mask_list[rand[8]]]]
    mask_im = Image.new('RGB', (240, 240))
    t_offset = 0
    m_offset = 0
    b_offset = 0
    for im in top_images:
        mask_im.paste(im, (t_offset,0))
        t_offset += im.size[0]
    for im in mid_images:
        mask_im.paste(im, (m_offset,80))
        m_offset += im.size[0]
    for im in bot_images:
        mask_im.paste(im, (b_offset,160))
        b_offset += im.size[0]
    mask_im.save(out_mask_dir+'\\'+str(iter_num)+'.png')
    iter_num+=1
